In [1]:
from pyomo.environ import *
import math

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
#this function prints the results of the solver status of the program
def solverstatus(results):
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        print('feasible')
    elif (results.solver.termination_condition == TerminationCondition.infeasible):
        print('infeasible')
    else:
        print ('Solver Status:',  results.solver.status)
#this function adds 2 lists together and removes the identical elements
def remove_dup(first_list,second_list):
    return(list(dict.fromkeys(first_list + second_list)))   

    
model = AbstractModel()
model.i = Set(doc='set of Gen units')
model.b = Param(model.i, doc='b coef of cost')
model.pmin = Param(model.i, doc='Min gen')
model.pmax = Param(model.i, doc='Max gen')
model.t = Set(doc='set of time steps')
model.dem = Param(model.t, doc='System demand', mutable=True)
model.GBconnect = Set(dimen=2, doc='Generator connected to bus')

model.lines = Set(dimen=3,doc = "Lines connecting bus and node")
model.branchX = Param(model.lines, doc='Max gen')
model.branchLim = Param(model.lines, doc='Max gen')

model.bus = Set()
model.node = Set(initialize=model.bus)

model.sbase = Param(doc='Sbase')
model.Pd = Param(model.bus, doc='Max gen')
model.delta = Var(model.bus,model.t, bounds=(-math.pi, math.pi), domain=Reals, doc='angle')

#Defines the Pmin and Pmax of the busses
def Pbounds(model,i,t):
     return (0*model.pmin[i]/model.sbase, model.pmax[i]/model.sbase)
model.P = Var(model.i,model.t, bounds=Pbounds, domain=Reals)

#Objective Function for Bus and nodes.
def objective_function_rule(model):
    return sum(model.b[i]*model.P[i,t] for i in model.i for t in model.t)
model.objective_function = Objective(rule=objective_function_rule, sense = minimize)

def LineBounds(model,lines,bus,node,t):
    
    if (lines,bus,node)  in model.lines:

        return(-model.branchLim[lines,bus,node]/model.sbase,model.branchLim[lines,bus,node]/model.sbase)
    else:
        return Constraint.Skip    
model.flow = Var( model.lines,model.t,bounds = LineBounds , domain=Reals, doc='flow')



#this function specifies which bus is attached to which nodes. [correct]
def NodesOut_init(model, bus):
    
    retval1 = []
    retval2 = []
    retval = []
    for (l,i,j)  in model.lines:
        if i == bus and j != bus:
            #print(i,j,bus)
            retval1.append(j)
    for (l,j,i)  in model.lines:
        if i == bus and j != bus:
            retval2.append(j)
    retval.append(remove_dup(retval1,retval2))
    #print(str(retval))
    return retval
model.NodesOut = Set(model.bus, initialize=NodesOut_init,doc='outgoing flow nodes')


#this function will specify which generator is attached to what bus. [correct]
def Nodeswithgen(model, bus):
    retval = []
    for (gen,node) in model.GBconnect:
        if node == bus:
            retval.append(gen)
    #print(retval) 
    return retval
model.GB = Set(model.bus, initialize=Nodeswithgen,doc='Nodes with generator')

#Definies the flow on each line, this function has the same issue of "LineBounds" function where it works but only for the buses and nodes provided. 
def linecalc_rule(model,line,bus,node,t):
    if (line,bus,node) in model.lines:
        #print("y")
        return model.flow[line,bus,node,t] == (1/model.branchX[line,bus,node])*(model.delta[bus,t] - model.delta[node,t])
    else:
        return Constraint.Skip
model.constflowcalc = Constraint(model.lines, model.t,rule=linecalc_rule)


#defines the balance rule on each bus 
def balance_rule(model,bus,t):
    return sum(model.P[i,t] for i in model.GB[bus]) - model.dem[t]*model.Pd[bus]/model.sbase == sum(model.flow[line,b,node,t] for (line,b,node) in model.lines if b == bus)
    

model.constbalance = Constraint(model.bus,model.t , rule=balance_rule)


opt = SolverFactory('glpk')
instance = model.create_instance("data3_t.dat")
results = opt.solve(instance)


In [4]:
#solverstatus(results)

In [5]:
#instance.pprint()

In [6]:
# print("---System Data---")
# print("--Power Demand at each bus--")
# for pd in instance.bus:
#      print(pd , instance.Pd[pd])
# print("--Branch Limits--")
# for bl in instance.lines:
#      print(bl , instance.branchLim[bl])
# print("--Line Impeadance--")
# for imp in instance.lines:
#      print(imp , instance.branchX[imp])

In [8]:
# print("---Results---")
# print(value(instance.objective_function.expr)*instance.sbase)
PowerGen = []
for i in instance.i:
    for t in instance.t:
        PowerGen.append([i , t, instance.P[i, t].value])
# for [g,t,p] in PowerGen:
#     print(g,t,p)
    
    
LineFlow = [] 
for [l,b,n,t] in instance.flow:
    if instance.flow[l,b,n,t].value >= 0:
        LineFlow.append([l,b,n,t,instance.flow[l,b,n, t].value])
for [l,b,n,t,F] in LineFlow:
     print(l,b,n,t,F)

    
DeltaValue = []
for [a,t] in instance.delta:
    DeltaValue.append([a,t,instance.delta[a,t].value])
# for [a,t,F] in DeltaValue:
#     print(a,t,F)





L1 b1 b2 1 2.30770865507285
L1 b1 b2 2 2.16525750352514
L1 b1 b2 3 2.49716765042727
L2 b1 b4 1 1.27955966243412
L2 b1 b4 2 1.20057449808634
L2 b1 b4 3 1.8678838868822
L4 b2 b3 1 0.606708655072849
L4 b2 b3 2 0.569257503525143
L3 b5 b1 1 3.58726831750697
L3 b5 b1 2 3.36583200161148
L3 b5 b1 3 2.26505153730947
L4 b3 b2 3 0.502832349572732
L5 b4 b3 1 1.09429134492715
L5 b4 b3 2 1.02674249647486
L5 b4 b3 3 0.267883886882217
L6 b5 b4 1 2.082731682493
L6 b5 b4 2 1.95416799838849
L6 b5 b4 3 2.39999999999998
